In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime

In [2]:
# Starting URL for Weather Map API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude & longitude coordinates
lats = np.random.uniform(-90.000, 90.000, size=1500)
lngs = np.random.uniform(-180.000, 180.000, size=1500)
# Zip together lat/long combos
lat_lngs = zip(lats, lngs)    # packs each pair of lats & lngs having the same index into a TUPLE
lat_lngs

In [4]:
# Save lat/long combos as a list
coordinates = list(lat_lngs)

In [5]:
# Create a list to hold cities we've identified
cities = []

# Identify nearest city for each lat/long combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Add city to cities list - but only if it is not already included
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count
len(cities)

614

In [ ]:
# Initialize empty lists to hold weather data
city_data = []

# Print the beginning of the logging.
print('Beginning Data Retrieval          ')
print('----------------------------------')

# Create counters
record_count = 1
set_count = 1

# Loop through all cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.
    city_url = url + '$q=' + city.replace(' ','+')
    
    # Log the URL, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    # Add 1 to record count
    record_count += 1

Beginning Data Retrieval          
----------------------------------
Processing Record 1 of Set 1 | nome
Processing Record 2 of Set 1 | humaita
Processing Record 3 of Set 1 | north platte
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | port macquarie
Processing Record 6 of Set 1 | hobart
Processing Record 7 of Set 1 | sidney
Processing Record 8 of Set 1 | rafraf
Processing Record 9 of Set 1 | gillette
Processing Record 10 of Set 1 | rafaela
Processing Record 11 of Set 1 | grindavik
Processing Record 12 of Set 1 | mitsamiouli
Processing Record 13 of Set 1 | punta arenas
Processing Record 14 of Set 1 | labytnangi
Processing Record 15 of Set 1 | hermanus
Processing Record 16 of Set 1 | aklavik
Processing Record 17 of Set 1 | noumea
Processing Record 18 of Set 1 | qaanaaq
Processing Record 19 of Set 1 | vagur
Processing Record 20 of Set 1 | lakatoro
Processing Record 21 of Set 1 | herat
Processing Record 22 of Set 1 | yellowknife
Processing Record 23 of Set 1 | bam